In [70]:
# Temporal Aatching model between Users and Locations
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
import calendar
from datetime import datetime
from datetime import timedelta
from scipy import sparse
from scipy.stats import norm
from numpy.random import *
from scipy import optimize

In [71]:
# 連続した日付を取得する関数
def daterange(_start, _end):
    for n in range((_end - _start).days):
        yield _start + timedelta(n)
        
# 切断ポアソン分布を生成する関数
def rtpois(mu, a, b, n):
    FA = scipy.stats.poisson.cdf(a, mu)
    FB = scipy.stats.poisson.cdf(b, mu)
    return np.array(scipy.stats.poisson.ppf(np.random.uniform(0, 1, n)*(FB-FA)+FA, mu), dtype="int")

# 多項分布の乱数を生成する関数
def rmnom(pr, n, k, pattern):
    if pattern==1:
        z_id = np.array(np.argmax(np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis], axis=1), dtype="int")
        Z = np.diag(np.repeat(1, k))[z_id, ]
        return z_id, Z
    z_id = np.array(np.argmax((np.cumsum(pr, axis=1) >= np.random.uniform(0, 1, n)[:, np.newaxis]), axis=1), dtype="int")
    return z_id

In [72]:
# シミュレーションデータを生成
# 日付データを作成
# 対象期間の日付と曜日情報を出力
start = datetime.strptime('2014-01-01', '%Y-%m-%d')
end   = datetime.strptime('2019-09-30', '%Y-%m-%d')
get_date = np.array([])
day_of_week = np.array([], dtype="int")
for day in daterange(start, end):
    get_date = np.append(get_date, day)
    day_of_week = np.append(day_of_week, day.weekday())
    
#日付を定義
index_week = np.array(np.where(day_of_week < 5)[0], dtype="int")
Timestamp = pd.Series(get_date).iloc[index_week]
date = pd.DataFrame({"date": Timestamp.astype("U")})
date_df = pd.merge(date, pd.DataFrame({"date": np.unique(date), "date_id": np.arange(date.shape[0])}), on="date", how="inner")
year = pd.DataFrame({"year": Timestamp.astype("U").str[:4]})
unique_year = np.unique(year)
year_df = pd.merge(year, pd.DataFrame({"year": unique_year, "year_id": np.arange(unique_year.shape[0])}), on="year", how="inner")
month = pd.DataFrame({"month": Timestamp.astype("U").str[5:7]})
unique_month = np.unique(month)
month_df = pd.merge(month, pd.DataFrame({"month": unique_month, "month_id": np.arange(unique_month.shape[0])}), on="month", how="inner")

#週を定義
new_week = day_of_week[index_week]
index = np.array(np.where((new_week-4)==0)[0], dtype="int")
week_n = index.shape[0]
week = np.repeat(0, new_week.shape[0])
for i in range(week_n):
    if i==0:
        get = np.arange(0, index[i]+1)
        week[get] = np.repeat(i, get.shape[0])
    else:
        get = np.arange(index[i-1]+1, index[i]+1)
        week[get] = np.repeat(i, get.shape[0])
        
#データフレームを作成
calendar_df = pd.concat((date_df, year_df, month_df, pd.DataFrame({"week_id": week})), axis=1)
date = np.array(calendar_df["date_id"], dtype="int")
year = np.array(calendar_df["year_id"], dtype="int")
month = np.array(calendar_df["month_id"], dtype="int")
week = np.array(calendar_df["week_id"], dtype="int")

In [73]:
# データの定義
k = 10
hh = 5000
place = 3000
category = 50
area = 25
Lambda = np.random.gamma(40.0, 1/0.3, hh)
pt = np.random.poisson(Lambda, hh)
hhpt = np.sum(pt)
k_vec = np.repeat(1.0, k)

In [74]:
# idとインデックスを定義
# idの定義
d_id = np.repeat(np.arange(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスを定義
d_list = [i for i in range(hh)]
for i in range(hh):
    d_list[i] = np.array(np.where(d_id==i)[0], dtype="int")

In [43]:
# placeを選択
#トピックを生成
topic = 30 
theta_topic = np.random.dirichlet(np.repeat(0.2, topic), hh)
phi_place = np.random.dirichlet(np.repeat(0.2, place), topic)
z = np.array(rmnom(theta_topic[d_id, ], hhpt, topic, 0), dtype="int16")

# 多項分布からplaceを生成
place_id = np.repeat(0, hhpt)
for i in range(hh):
    index = d_list[i]
    place_id[index] = rmnom(phi_place[z[index], ], pt[i], place, 0)
    
# インデックスを定義
place_list = [i for i in range(place)]
place_n = np.repeat(0, place)
for i in range(place):
    place_list[i] = np.array(np.where(place_id==i)[0], dtype="int")
    place_n[i] = place_list[i].shape[0]

In [81]:
# areaとcategoryを選択
# トピックを生成
theta_place = np.random.dirichlet(np.repeat(0.3, topic), 1).reshape(-1)
phi_area = np.random.dirichlet(np.repeat(0.2, area), topic)
phi_category = np.random.dirichlet(np.repeat(0.2, category), place)
s = np.array(np.argmax(np.random.multinomial(1, theta_place, place), axis=1), dtype="int")

# 多項分布からareaとcategoryを生成
area_id = rmnom(phi_area[s, ], place, area, 0)[place_id]
category_id = rmnom(phi_category[s, ], place, category, 0)[place_id]

# インデックスを定義
area_list = [i for i in range(place)]
category_list = [i for i in range(category)]
area_n = np.repeat(0, area)
category_n = np.repeat(0, category)
for i in range(area):
    area_list[i] = np.array(np.where(area_id==i)[0], dtype="int")
    area_n[i] = area_list[i].shape[0]
for i in range(category):
    category_list[i] = np.array(np.where(category_id==i)[0], dtype="int")
    category_n[i] = category_list[i].shape[0]

In [54]:
# 日付と時間を割り当て
# トピックを生成
weekday = 2
hours = 24
phi_weekday = np.random.beta(3.0, 5.0, place)
phi_hours = np.random.dirichlet(np.repeat(0.1, hours), place)
z = np.array(rmnom(theta_topic[d_id, ], hhpt, topic, 0), dtype="int16")

# 多項分布から日付と時間を生成
day_id = np.repeat(0, hhpt)
hours_id = np.repeat(0, hhpt)
for i in range(hh):
    index = d_list[i]
    day_id[index] = np.random.binomial(1, phi_weekday[z[index]], pt[i])
    hours_id[index] = rmnom(phi_hours[z[index], ], pt[i], hours, 0)

In [56]:
hours_id

array([ 3,  1, 18, ..., 18, 18, 16])